# BTTH08: Regularized Neural Net

TODO: Ghi họ tên và MSSV của bạn (vd, Nguyễn Văn A - 1234567)

---

Nguyễn Phan Mạnh Hùng - 1312727


## 1. Cách làm bài và nộp bài

**Làm bài**

Bạn sẽ làm trực tiếp trên file notebook này; trong file, mình đã để từ `TODO` để cho biết những chỗ mà bạn cần phải làm (trong đó, `TODO` đầu tiên là bạn phải ghi họ tên và MSSV vào phần đầu của file). Trong khi làm bài, thường xuyên `Ctrl + S` để lưu lại bài làm của bạn, tránh mất mát thông tin.

*Lưu ý: tuyệt đối không gian lận. Nếu vi phạm thì bạn sẽ bị 0 điểm cho cả phần thực hành môn học. Nên nhớ mục tiêu chính ở đây là học kiến thức.*

**Nộp bài**

Khi chấm bài, đầu tiên mình sẽ chọn `Cell` - `Run All` để chạy tất cả các cell trong notebook của bạn; do đó, trước khi nộp bài, bạn nên chạy thử `Cell` - `Run All` để đảm bảo mọi chuyện diễn ra đúng như mong đợi.

Sau đó, trong thư mục `MSSV` (vd, nếu bạn có MSSV là 1234567 thì bạn đặt tên thư mục là `1234567`) bạn đặt file `Ex08-RegularizedNeuralNet.ipynb` (không cần nộp file dữ liệu `mnist.pkl.gz`); rồi nén thư mục `MSSV` này lại và nộp ở link trên moodle.

## 2. Import

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cPickle
import gzip
# You can also import other things ...
import copy

## 3. Hàm đọc dữ liệu

Trong bài này, bạn sẽ thử nghiệm Neural Net trên bộ dữ liệu MNIST (file `mnist.pkl.gz` đính kèm). Đây là bộ dữ liệu gồm các ảnh chữ số viết tay từ 0-9 (10 lớp); mỗi ảnh có kích thước $28\times 28$ và là ảnh grayscale. Bộ dữ liệu đã được chia sẵn làm 3 tập: tập huấn luyện gồm 50000 ảnh, tập validation gồm 10000 ảnh, và tập kiểm tra gồm 10000 ảnh.

In [2]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.plk.gz').
    
    Returns
    -------
    (X_train, Y_train, X_val, Y_val, X_test, Y_test) : tuple
        X_train : numpy array, shape (N=50000, d+1=785)
            Input vectors of the training set.
        Y_train: numpy array, shape (N=50000)
            Outputs of the training set.
        X_val : numpy array, shape (N=10000, d+1=785)
            Input vectors of the validation set.
        Y_val: numpy array, shape (N=10000)
            Outputs of the validation set.
        X_test : numpy array, shape (N=10000, d+1=785)
            Input vectors of the test set.
        Y_test: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = cPickle.load(f)
    f.close()
    
    X_train, Y_train = train_data
    X_train = np.hstack((np.ones((X_train.shape[0], 1)), X_train))
    
    X_val, Y_val = val_data
    X_val = np.hstack((np.ones((X_val.shape[0], 1)), X_val))
    
    X_test, Y_test = test_data
    X_test = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
    
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

## 4. Hàm lan truyền tiến qua Neural Net

Trong bài này, ta sẽ sử dụng nơ-ron sigmoid ở các tẩng ẩn, và tầng softmax là tầng xuất.

In [3]:
def sigmoid(Z):
    """
    Computes sigmoid function for each element of numpy array Z.
    """
    return 1 / (1 + np.exp(-Z))

def softmax(Z):
    """
    Computes softmax function for each row of numpy array Z.
    """
    A = np.exp(Z)
    A /= np.sum(A, axis=1, keepdims=True)
    return A

def forward_prop(X, Ws):
    """
    Forward propagates X through layers of neural nets to get the final outputs.
    
    Parameters
    ----------
    X : numpy array, shape (N, d+1)
        The matrix of input vectors (each row corresponds to an input vector); the first column of 
        this matrix is all ones (corresponding to x_0).
    Ws : list of numpy arrays
        The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
        d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
        d^(l) is the number of neurons (not count the +1 neuron) of layer l.
    
    Returns
    -------
    A : numpy array, shape (N, K=10)
        The maxtrix of Neural Net's output vectors; each row is an output vector (containing each 
        class's probability given the corresponding input vector).
    """
    # TODO
    A = X;
    for i in range(len(Ws)):
        W = Ws[i];
        Z = A.dot(W)
        if i == len(Ws)-1:
            A = softmax(Z);
        else:
            A = sigmoid(Z);
            A = np.hstack((np.ones((A.shape[0],1)),A));
    return A;
      

## 5. Hàm huấn luyện Neural Net

In [4]:
def addOne(A):
    return np.hstack((np.ones((A.shape[0],1)),A));
def meanBinaryError(X, W, Y):
    #for debugging: def forward_prop(X, Ws):
    A = forward_prop(X, W);
    myY = np.nonzero(np.max(A, axis = 1, keepdims = True) == A)[1]
    error = 1 - np.sum(myY == Y) * 1.0 / Y.shape[0]
    return error
def train_neural_net(X_train, Y_train, X_val, Y_val, layer_sizes, learning_rate, mnb_size, max_patience, 
                     l2_reg_level):
    """
    Trains Neural Net on the dataset (X_train, Y_train).
    Cost function: Mean Negative Log Likelihood + L2 regularization.
    Optimization algorithm: Stochastic Gradient Descent (SGD) with early stopping.
    
    Parameters
    ----------
    X_train : numpy array, shape (N, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the training set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_train : numpy array, shape (N,)
        The vector of outputs of the training set.
    X_val : numpy array, shape (N_val, d + 1)
        The matrix of input vectors (each row corresponds to an input vector) of the validation set; 
        the first column of this matrix is all ones (corresponding to x_0).
    Y_val : numpy array, shape (N_val,)
        The vector of outputs of the validation set.  
    layer_sizes : list of ints
        The list of each layer' size (not count the +1 neurons).
        E.g. layer_sizes = [784, 30, 10] means: the 1st layer (input layer) has 784 neurons,
        the 2nd layer (hidden layer) has 30 neurons, the 3rd layer (output layer) has 10 neurons.
    learning_rate : float
        Learning rate of SGD.
    mnb_size : int
        Minibatch size of SGD.
    max_patience : int
        The parameter of early stopping. You'll have a `patience` variable with initial value equal to
        `max_patience`. During the training, you'll keep track of the best MBE (Mean Binary Error) 
        on the validation set; if the MBE on the validation set at the current epoch < the current 
        best one, you'll reset `patience` to `max_patience`; otherwise, `patience` -= 1. 
        When `patience` = 0, you'll terminate SGD.
    l2_reg_level : float
        The level (the coefficient) of L2 regularization.
    
    Returns
    -------
    (best_Ws, train_errs, val_errs) : tuple
        best_Ws : list of numpy arrays
            The list of each layer's W; W of layer l will have the shape of (d^(l-1)+1, d^(l)) where 
            d^(l-1) is the number of neurons (not count the +1 neuron) of layer l-1, and 
            d^(l) is the number of neurons (not count the +1 neuron) of layer l.
            It's the parameters having smallest MBE on the validation set.
        train_errs: list of floats
            List of MBEs on the training set after each epoch.
        val_errs: list of floats
            List of MBEs on the validation set after each epoch.
    
    Notes
    -----
    After each epoch, you need to print out: 
    - The MBE on the training set and validation set.
    - The value of `patience`.
    E.g., 'Epoch ..., training err ..., val err ..., patience ...'.
    
    After the training, you need to print out:
    - The best MBE on the validation set.
    - The corresponding epoch.
    - The corresponding MBE on the training set.
    E.g., "Best val err ..., at epoch ..., corresponding train err ...".
    """
    # Init weights
    np.random.seed(0) # Note: this will fix the randomization so that we'll get the same random numbers each run; 
                      # it make comparisons (e.g. between different values of `l2_reg_level`) more accurate. 
    Ws = [np.random.randn(layer_sizes[l]+1, layer_sizes[l+1]) / np.sqrt(layer_sizes[l]+1) for l in range(len(layer_sizes)-1)]
    
    # TODO
    bestMBEval = 111; #infinity ~ > 1
    corEpoch = -1;
    corMBEtrain = 111;
    train_errs = [];
    val_errs = [];
    bestWs = [];
    
    N = X_train.shape[0];
    one_hot_Y = np.eye(layer_sizes[-1]);
    rand_idxs = range(N);
    max_epoch = 1000000000; #infinity 
    epoch = -1;
    while epoch < max_epoch: #we can also use "While True:". However, we should use this condition instead to control when the loop terminates. 
        epoch += 1
        np.random.shuffle(rand_idxs);
        for start_idx in range(0,N, mnb_size):
            mnb_X = X_train[rand_idxs[start_idx:start_idx+mnb_size]];
            mnb_Y = one_hot_Y[Y_train[rand_idxs[start_idx:start_idx+mnb_size]]];
            #forward propagation
            As = [mnb_X];
            #Zs =[];
            A = mnb_X;
            for i in range(len(Ws)):
                W = Ws[i];
                Z = A.dot(W)
                if i == len(Ws)-1:
                    A = softmax(Z);
                else:
                    A = sigmoid(Z);
                    A = np.hstack((np.ones((A.shape[0],1)),A));
                    
                #Zs.append(Z); #for debuging
                As.append(A);
            #backward propagation
            delta = As[-1] - mnb_Y;
            grad = As[-2].T.dot(delta)*1.0/mnb_size + 2.0*l2_reg_level*Ws[-1]; #/mnb_size
            Ws[-1] -= learning_rate*grad;
            
            for it in range(2, len(layer_sizes)):
                #print delta.shape, Ws[-it+1].shape, it
                
                delta = delta.dot(Ws[-it+1].T)*As[-it]*(1-As[-it])
                grad = (As[-it-1].T.dot(delta)*1.0/mnb_size)[:,1:] + 2.0*l2_reg_level*Ws[-it]; #/mnb_size
                #grad = grad[:,1:] + 1.0*l2_reg_level*Ws[-it];
                #if it == 2 and epoch == 0:
                    #print (grad[:,1:]).shape, (Ws[-2]).shape
                Ws[-it] -= learning_rate*grad;
                delta = delta[:,1:];
        
        eTrain = meanBinaryError(X_train, Ws, Y_train);
        eVal = meanBinaryError(X_val, Ws, Y_val);
        train_errs.append(eTrain);
        val_errs.append(eVal);
        
        if (eVal < bestMBEval):
            bestMBEval = eVal;
            corMBEtrain = eTrain;
            corEpoch = epoch
            patience = max_patience
            bestWs = copy.deepcopy(Ws);
        else:
            patience = patience - 1;        
        print 'Epoch ', epoch, ', training err ', eTrain*100, '%, val err ', eVal*100, '%, patience ', patience, '\n'
        if patience == 0:
            break;
            
       
    print 'Best val err ', bestMBEval*100, '% at epoch ', corEpoch, ' corresponding train err ',corMBEtrain*100, '%';
    return (bestWs,train_errs,val_errs);

## 6. Thí nghiệm

Để thấy được ảnh hưởng của hệ số `l2_reg_level`, bạn sẽ dùng các hàm đã định nghĩa ở trên như sau:

1. Đọc dữ liệu.
2. Huấn luyện Neural Net trên tập huấn luyện với `layer_sizes = [784, 30, 10]`, `learning_rate = 0.1`, `mnb_size = 10`, `max_patience = 20`, và `l2_reg_level = 0, 0.0001, 0.001`. Để dễ nhìn khi chương trình `print` ra, bạn nên dùng 3 code cell cho 3 lần gọi hàm huấn luyện (ứng với 3 giá trị của `l2_reg_level`).
3. Ở cell kế tiếp, bạn sẽ vẽ ra đồ trị có trục hoàng là số lượng epoch và trục tung là độ lỗi. Với mỗi giá trị của `l2_reg_level`, bạn sẽ vẽ ra 2 đường ứng với độ lỗi MBE trên tập huấn luyện và tập validation; như vậy, trên đồ thị sẽ có tất cả 6 đường.
4. Cho nhận xét dựa vào đồ thị kết quả.
5. Cuối cùng, bạn sẽ tính và in ra độ lỗi trên tập kiểm tra của mô hình có độ lỗi nhỏ nhất trên tập validation (trong số 3 mô hình ứng với 3 giá trị của `l2_reg_level`).

(Kết quả chạy của mình: với `l2_reg_level = 0`, độ lỗi trên tập huấn luyện và tập validation lần lượt là 1.076% và 3.480%; với `l2_reg_level = 0.0001`, độ lỗi là 2.114% và 2.910%; với `l2_reg_level = 0.001`, độ lỗi là 6.940% và 6.130%.)

In [5]:
# TODO
X_train, Y_train, X_val, Y_val, X_test, Y_test = read_mnist('mnist.pkl.gz');
l2_reg = [0, 0.0001, 0.001];
bestWs = [];
train_errs = [];
val_errs = [];
#def train_neural_net(X_train, Y_train, X_val, Y_val, layer_sizes, learning_rate, mnb_size, max_patience, l2_reg_level):


In [6]:
middle = [1, 2, 5, 15, 20, 30, 40, 50, 100]
for mid in middle:
    bWs,tE,vE = train_neural_net(X_train, Y_train, X_val, Y_val, [784,mid,10], 0.1, 10, 20, 0.0001);
    bestWs.append((bWs, mid));
    train_errs.append(tE);
    val_errs.append(vE);

for WS, mid in bestWs:
    print  'Hidden layer ', mid,  '. Test error ', meanBinaryError(X_test, WS, Y_test)*100, '%';
    

Epoch  0 , training err  70.88 %, val err  71.19 %, patience  20 

Epoch  1 , training err  71.626 %, val err  72.02 %, patience  19 

Epoch  2 , training err  69.412 %, val err  70.31 %, patience  20 

Epoch  3 , training err  70.444 %, val err  70.92 %, patience  19 

Epoch  4 , training err  73.884 %, val err  73.73 %, patience  18 

Epoch  5 , training err  70.552 %, val err  71.3 %, patience  17 

Epoch  6 , training err  68.748 %, val err  68.82 %, patience  20 

Epoch  7 , training err  71.282 %, val err  71.89 %, patience  19 

Epoch  8 , training err  68.748 %, val err  68.89 %, patience  18 

Epoch  9 , training err  71.39 %, val err  71.46 %, patience  17 

Epoch  10 , training err  69.06 %, val err  68.93 %, patience  16 

Epoch  11 , training err  69.848 %, val err  70.51 %, patience  15 

Epoch  12 , training err  68.778 %, val err  68.46 %, patience  20 

Epoch  13 , training err  67.63 %, val err  67.34 %, patience  20 

Epoch  14 , training err  68.682 %, val err  68.1

- Ta nhận thấy độ lỗi trên tập train và validation ứng với l2_reg_level = 0.001 đều cao, và cao hơn so với các giá trị l2 khác. Điều này cho thấy mô hình này bị underfiting do giá trị l2 khá cao dẫn đến việc các giá trị Ws tương ứng không được "dao động thoải mái" và có xu hướng gần giá trị 0 để tối ưu hàm chi phí J do đó khó đạt được tới mô hình tối ưu.
- Ngược lại với giá trị l2 = 0, thì mô hình này có độ lỗi trên tập train xuống rất thấp nhưng đồng thời độ lỗi trên tập validation ban đầu có xu hướng xuống nhưng sau lại tăng lên. Điều này cho thấy mô hình này bị overfiting do giá trị l2 quá thấp dẫn tới việc các giá trị Ws 'thoải mái di chuyển' và cố gắng fit tập train (gồm cả nhiễu) dẫn tới việc mô hình không được tổng quát.
- Còn với giá trị l2 = 0.001 thì ta thấy độ lỗi trên tập train và tập validation đều có xu hướng đi xuống và có giá trị thấp. Mô hình này tương đối tốt nếu so với 2 mô hình còn lại do giá trị l2 được chọn khá cân bằng (không quá cao hay thấp).

In [7]:
layer = [[784, 50, 30, 10], [784, 50, 30, 20, 10], [784, 50, 30, 20, 20, 10]]
for ll in layer:
    bWs,tE,vE = train_neural_net(X_train, Y_train, X_val, Y_val, ll, 0.1, 10, 20, 0.0001);
    bestWs.append((bWs, ll));
    train_errs.append(tE);
    val_errs.append(vE);

for WS, ll in bestWs:
    print ll 
    print  '. Test error ', meanBinaryError(X_test, WS, Y_test)*100, '%';

Epoch  0 , training err  9.082 %, val err  8.62 %, patience  20 

Epoch  1 , training err  6.254 %, val err  5.85 %, patience  20 

Epoch  2 , training err  4.692 %, val err  4.63 %, patience  20 

Epoch  3 , training err  3.832 %, val err  3.98 %, patience  20 

Epoch  4 , training err  3.674 %, val err  4.17 %, patience  19 

Epoch  5 , training err  2.99 %, val err  3.65 %, patience  20 

Epoch  6 , training err  3.24 %, val err  3.82 %, patience  19 

Epoch  7 , training err  2.518 %, val err  3.17 %, patience  20 

Epoch  8 , training err  2.308 %, val err  3.08 %, patience  20 

Epoch  9 , training err  2.342 %, val err  3.28 %, patience  19 

Epoch  10 , training err  2.2 %, val err  3.01 %, patience  20 

Epoch  11 , training err  2.264 %, val err  3.09 %, patience  19 

Epoch  12 , training err  1.862 %, val err  2.64 %, patience  20 

Epoch  13 , training err  1.956 %, val err  2.78 %, patience  19 

Epoch  14 , training err  1.942 %, val err  2.76 %, patience  18 

Epoch  15

In [11]:
bWs,tE,vE = train_neural_net(X_train, Y_train, X_val, Y_val, [784,200,10], 0.1, 10, 20, 0.0001);
print  'Hidden layer ', mid,  '. Test error ', meanBinaryError(X_test, bWs, Y_test)*100, '%';

Epoch  0 , training err  8.524 %, val err  8.02 %, patience  20 

Epoch  1 , training err  6.382 %, val err  5.9 %, patience  20 

Epoch  2 , training err  4.852 %, val err  4.58 %, patience  20 

Epoch  3 , training err  4.266 %, val err  3.97 %, patience  20 

Epoch  4 , training err  4.536 %, val err  4.54 %, patience  19 

Epoch  5 , training err  3.284 %, val err  3.46 %, patience  20 

Epoch  6 , training err  4.018 %, val err  3.97 %, patience  19 

Epoch  7 , training err  2.768 %, val err  3.18 %, patience  20 

Epoch  8 , training err  2.854 %, val err  3.22 %, patience  19 

Epoch  9 , training err  2.24 %, val err  2.73 %, patience  20 

Epoch  10 , training err  2.466 %, val err  2.93 %, patience  19 

Epoch  11 , training err  2.402 %, val err  2.97 %, patience  18 

Epoch  12 , training err  2.586 %, val err  3.09 %, patience  17 

Epoch  13 , training err  2.074 %, val err  2.67 %, patience  20 

Epoch  14 , training err  2.122 %, val err  2.72 %, patience  19 

Epoch  